<a href="https://colab.research.google.com/github/Shashank-Ojha/WordFormer/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [174]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from utils.data_pipelines import read_file
from utils.data_pipelines import get_vocab
from utils.data_pipelines import train_validation_split
from utils.data_pipelines import get_batch

from utils.gpt import GPTConfig
from utils.gpt import GPT

from utils.loss import loss_fn
from utils.loss import estimate_loss

# Set a seed for reproducibility
torch.manual_seed(0)

In [175]:
# Read text file, convert each token (char) into integers, and split data into train and validation data.
text = read_file('shakespeare.txt')

# Here are all the unique characters that occur in this text
vocab, vocab_size = get_vocab(text)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
train_data, val_data = train_validation_split(data, 0.9)

In [176]:
print(data.shape)

torch.Size([1115394])


In [190]:
# ------ Hyperparameters ------

# -- Data Breakdown
batch_size = 32
block_size = 8

# -- Network Parameters
# Embedding Block
num_embeddings = vocab_size
embedding_dim = 8

# Attention Block
kq_dim = 7
v_dim = 6

# -- Training Parameters
learning_rate = 3e-4
max_iters = 50000
eval_iters = 1000

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [178]:
# Both x and y are (batch_size, block_size)
x, y = get_batch(train_data, batch_size, block_size)

In [187]:
# -- Define Network
config_dict = {
    'vocab_size': vocab_size,
    'max_seq_length': block_size,
    'embed_dim': embedding_dim,
    'kq_dim': kq_dim,
    'v_dim': v_dim,
}

config = GPTConfig(**config_dict)
print(config)

# # Input Shape (batch_size, block_size). 
# # Note that while block_size is passed to the Self Attention module, it really is
# # just an upper bound on the max block size. Any input shape of (batch_size, t) where
# # t <= block_size is valid.
model = GPT(config)

# Expected output (batch_size, block_size, vocab_size)
print(model(x).shape)

GPTConfig(vocab_size=65, max_seq_length=8, embed_dim=8, kq_dim=7, v_dim=6)
torch.Size([16, 8, 65])


In [191]:
# -- Training

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training Loop
for i in range(max_iters):
    # Shape = (batch_size, block_size)
    xb, yb = get_batch(train_data, batch_size, block_size)
    
    # Zero out the gradients
    optimizer.zero_grad()
    
    # Shape = (batch_size, block_size, vocab_size)
    logits = model(xb)
    
    # Compute loss
    loss = loss_fn(logits, yb)
    loss.backward()

    # Adjust weights
    optimizer.step()
    
    # Gather data report
    if i % eval_iters == 0 or i == max_iters - 1:
        eval_interval
        losses = estimate_loss(model, train_data, val_data, eval_iters, batch_size, block_size)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        

step <built-in function iter>: train loss 2.5499, val loss 2.5585
step <built-in function iter>: train loss 2.5452, val loss 2.5535
step <built-in function iter>: train loss 2.5417, val loss 2.5470
step <built-in function iter>: train loss 2.5382, val loss 2.5477
step <built-in function iter>: train loss 2.5391, val loss 2.5508
step <built-in function iter>: train loss 2.5382, val loss 2.5474
step <built-in function iter>: train loss 2.5389, val loss 2.5445
step <built-in function iter>: train loss 2.5373, val loss 2.5452
step <built-in function iter>: train loss 2.5318, val loss 2.5439
step <built-in function iter>: train loss 2.5334, val loss 2.5403
step <built-in function iter>: train loss 2.5355, val loss 2.5393
step <built-in function iter>: train loss 2.5309, val loss 2.5441
step <built-in function iter>: train loss 2.5315, val loss 2.5388
step <built-in function iter>: train loss 2.5282, val loss 2.5441
step <built-in function iter>: train loss 2.5313, val loss 2.5354
step <buil

In [192]:
# Generate Shakespeare :)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
new_text = model.generate(context, max_new_tokens=500)
print(decode(new_text[0].tolist()))


Cas oss, tiw qurere e mde sfsrry. By

Toourr mi
I ne sdinde
Ad
KRERNLe,
FGENTuriee arnd t go
ANCeru-
PENRrdDan barllleverilowe prameme! Ys t bo Wtharow louh tedinl mo
Ad a hinsif wtea n en, lengid hisve celf hithe chomileou NISELLUCLORSyIOCa onds, I dre nyeu, thyeot mouepafo to wwure, m tidigp't bu freltr'beto nge touisirth heot ishye wee asqwh wind k'dl so ons st ory ourongam brer Wyonsu, rgs w cidHe:'s
Lilouny ngthom amelr te thaga MIpeo the
Thof bans!

I:
LKEZIA
CEUKLASEI'BXAmOUIREI tueduch y
